1. Instalirati neophodne pakete.

In [1]:
!pip install federpy deepface hnswlib numpy opencv-python IPython ipykernel IProgress tqdm tensorflow scikit-learn


[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install --upgrade ipywidgets


[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import cv2
import json
import pickle
import hnswlib
import numpy as np
from glob import glob
import tensorflow as tf
from deepface import DeepFace
from tqdm.notebook import tqdm
from federpy.federpy import FederPy
from sklearn.preprocessing import normalize

2. Ukoliko na uredjaju ne postoji graficka karta i instalirani CUDNN paketi, iskljuciti tensorflow pofrsku za GPU.

In [4]:
tf.config.set_visible_devices([], 'GPU')

3. Ucitati podatke koje cemo koristiti za kreiranje grafa.

In [5]:
extensions = ['*.JPEG', '*.jpg', '*.png', '*.jpeg']
to_insert = [item for ext in extensions for item in glob('./faces/*/' + ext)]

4. Primer predikcije jednog vektora obelezja.

Zamenom modela za enkodovanje slika i dataseta se vrlo jednostavno mozete prebaciti na resavanje nekog drugog problema.

In [6]:
embedding_objs = DeepFace.represent(img_path = "probe.png", model_name="Facenet512", enforce_detection=False)[0]['embedding']
print(len(embedding_objs))
print(embedding_objs)

512
[-1.1632728576660156, 0.9516001343727112, -0.7969666123390198, 1.2928544282913208, 2.5916643142700195, 1.2771377563476562, 0.05079638957977295, 0.17092810571193695, -0.3515681028366089, -0.44998374581336975, -0.9594532251358032, -0.3521721661090851, -0.5530012845993042, 0.437912255525589, 2.188519239425659, 1.1920498609542847, 1.5722578763961792, 3.2101776599884033, 0.011202633380889893, -0.05363209918141365, -0.9020317196846008, -1.5413823127746582, 0.045239564031362534, -0.2651774287223816, 0.37653017044067383, -1.3088165521621704, 0.2617620825767517, 1.248788595199585, 2.315866708755493, 0.4696466624736786, -0.8305005431175232, -2.200120687484741, -1.5008715391159058, -0.510016679763794, 0.8694159984588623, -0.24675416946411133, 1.81059730052948, -1.0802156925201416, -0.7808616757392883, 0.5048351883888245, 0.472255140542984, -0.1867733746767044, -0.4251719117164612, -2.104097604751587, 0.46141764521598816, -0.09510739147663116, 1.150063395500183, -0.22913411259651184, -0.604931

5. Normalizacija vektora na opseg 0-1.

In [7]:
embedding_objs = normalize([embedding_objs])[0]
print(len(embedding_objs))
print(embedding_objs)

512
[-4.71604249e-02  3.85789683e-02 -3.23099467e-02  5.24138114e-02
  1.05069064e-01  5.17766394e-02  2.05934429e-03  6.92962278e-03
 -1.42529769e-02 -1.82428607e-02 -3.88973417e-02 -1.42774663e-02
 -2.24193106e-02  1.77534685e-02  8.87250971e-02  4.83270779e-02
  6.37411498e-02  1.30144309e-01  4.54167693e-04 -2.17430723e-03
 -3.65694075e-02 -6.24894188e-02  1.83406417e-03 -1.07505992e-02
  1.52649679e-02 -5.30609344e-02  1.06121371e-02  5.06273317e-02
  9.38879106e-02  1.90400180e-02 -3.36694510e-02 -8.91954332e-02
 -6.08470653e-02 -2.06766651e-02  3.52471285e-02 -1.00036990e-02
  7.34037053e-02 -4.37931915e-02 -3.16570340e-02  2.04666015e-02
  1.91457687e-02 -7.57200828e-03 -1.72369603e-02 -8.53025466e-02
  1.87064042e-02 -3.85576347e-03  4.66248981e-02 -9.28936151e-03
 -2.45246108e-02  3.05993747e-02 -1.33360341e-02 -4.42943469e-02
 -2.36681637e-02 -5.24465494e-02 -3.47779016e-02 -6.14313898e-02
 -2.90362538e-02 -4.05152142e-02  7.57021710e-02  2.58167036e-02
 -5.62233840e-02  1.2

6. Kreiranje grafa uz pomoc HNSWLib paketa.

In [8]:
graph_data = {}
graph_name = 'mathverse'

In [9]:
graph = hnswlib.Index(space = 'cosine', dim = 512)
graph.init_index(max_elements = 100000, ef_construction = 100, M = 12)

Dodavanje normalizovanih vektora u graf.

Napomena: U HNSWLib paketu je moguce inkrementalno dodavati elemente i kada je graf vec napravljen. Drugi paketi (i biblioteke u drugim programskim jezicima) nemaju ovu mogucnost.

In [10]:
for id, data in enumerate(tqdm(to_insert)):
  vec = DeepFace.represent(img_path = data, model_name="Facenet512", enforce_detection=False)[0]['embedding']
  vec = normalize([vec])[0]
  graph.add_items(vec, id)
  graph_data[id] = data

  0%|          | 0/2375 [00:00<?, ?it/s]

Cuvanje grafa u fajl.

In [11]:
graph.save_index(graph_name)
with open('graph_data.json', 'wb') as f:
    f.write(json.dumps(graph_data, indent=4).encode('utf-8')) 

7. Ucitavanje grafa u FederPy i njegov prikaz.

In [12]:
# init federpy
hnswViewParams = {
    "width": 1000,
    "height": 700,
    "mediaType": "image",
    "mediaUrls": to_insert,
    "searchIntraLevelTime": 30,
}
federPy_hnsw = FederPy(graph_name, 'hnswlib', **hnswViewParams)
federPy_hnsw.overview(True)

8. Pretraga random test vektora

In [13]:

federPy_hnsw.searchRandTestVec()

9. Pretraga vektora po ID-ju.

In [14]:

federPy_hnsw.searchById(1224)

10. Pretraga na osnovu nove slika i vektora koji nije u grafu.

Vrednost k predstavlja najblize susede ovom vektoru koje zelimo da dobijemo. Postavljanjem ove vrednosti na 1, dobijemo prepoznavanje.
Postavljanjem na veci broj, dobijamo slicna lica.

In [15]:
vec = DeepFace.represent(img_path = "probe.png", model_name="Facenet512", enforce_detection=False)[0]['embedding']
vec = normalize([vec])[0]
federPy_hnsw.setSearchParams({"k": 1}).searchByVec(vec, 'probe.png')

In [32]:
ids, scores = graph.knn_query(vec, k = 1)
for id, score in zip(ids[0], scores[0]):
    print(id, score, graph_data[id])

1791 0.2160222 ./faces/4029/14.png


Isti rezultat mozemo dobiti i pretragom samog HNSW grafa.

In [17]:
graph.knn_query(vec, k = 1)

(array([[1791]], dtype=uint64), array([[0.2160222]], dtype=float32))

In [18]:
vec = DeepFace.represent(img_path = "probe.png", model_name="Facenet512", enforce_detection=False)[0]['embedding']
vec = normalize([vec])[0]
federPy_hnsw.setSearchParams({"k": 4}).searchByVec(vec, 'probe.png')

In [31]:
ids, scores = graph.knn_query(vec, k = 4)
for id, score in zip(ids[0], scores[0]):
    print(id, score, graph_data[id])

1791 0.2160222 ./faces/4029/14.png
1798 0.24477059 ./faces/4029/33.png
1778 0.26669413 ./faces/4029/29.png
1788 0.26900578 ./faces/4029/6.png


---